In [ ]:
-- Step 1: Create new database
CREATE DATABASE [PrestigeCars2.0];
GO

-- Step 2: Use the new database
USE [PrestigeCars2.0];
GO

-- Step 3: Create schema
CREATE SCHEMA Data;
GO

-- Step 4: Create all UDTs

-- COUNTRY
CREATE TYPE UDT_CountryID FROM INT;
CREATE TYPE UDT_CountryName FROM NVARCHAR(150);
CREATE TYPE UDT_CountryISO2 FROM NCHAR(10);
CREATE TYPE UDT_CountryISO3 FROM NCHAR(10);
CREATE TYPE UDT_Region FROM NVARCHAR(20);

-- MAKE
CREATE TYPE UDT_MakeID FROM INT;
CREATE TYPE UDT_MakeName FROM NVARCHAR(100);
CREATE TYPE UDT_MakeCountry FROM CHAR(3);

-- MODEL
CREATE TYPE UDT_ModelID FROM INT;
CREATE TYPE UDT_ModelName FROM NVARCHAR(150);
CREATE TYPE UDT_ModelVariant FROM NVARCHAR(150);
CREATE TYPE UDT_Year FROM CHAR(4);

-- CUSTOMER
CREATE TYPE UDT_CustomerID FROM NVARCHAR(5);
CREATE TYPE UDT_CustomerName FROM NVARCHAR(150);
CREATE TYPE UDT_Address FROM NVARCHAR(50);
CREATE TYPE UDT_IsReseller FROM BIT;
CREATE TYPE UDT_IsCreditRisk FROM BIT;

-- STOCK
CREATE TYPE UDT_StockCode FROM NVARCHAR(50);
CREATE TYPE UDT_ColorID FROM INT;
CREATE TYPE UDT_Cost FROM MONEY;
CREATE TYPE UDT_RepairsCost FROM MONEY;
CREATE TYPE UDT_PartsCost FROM MONEY;
CREATE TYPE UDT_TransportCost FROM MONEY;
CREATE TYPE UDT_BuyerComments FROM NVARCHAR(4000);
CREATE TYPE UDT_DateBought FROM DATE;
CREATE TYPE UDT_TimeBought FROM TIME(7);

-- SALES
CREATE TYPE UDT_SalesID FROM INT;
CREATE TYPE UDT_InvoiceNumber FROM CHAR(8);
CREATE TYPE UDT_SaleDate FROM DATETIME;
CREATE TYPE UDT_TotalSalePrice FROM NUMERIC(18,2);

-- SALES DETAILS
CREATE TYPE UDT_LineItemNumber FROM TINYINT;
CREATE TYPE UDT_SalePrice FROM NUMERIC(18,2);
CREATE TYPE UDT_LineItemDiscount FROM NUMERIC(18,2);
GO

-- Step 5: Create Tables

-- Country Table
CREATE TABLE Data.Country (
    CountryID UDT_CountryID PRIMARY KEY,
    CountryName UDT_CountryName NOT NULL DEFAULT N'Unknown',
    CountryISO2 UDT_CountryISO2 DEFAULT N'--',
    CountryISO3 UDT_CountryISO3 DEFAULT N'---',
    SalesRegion UDT_Region DEFAULT N'Unknown'
);
GO

-- Make Table
CREATE TABLE Data.Make (
    MakeID UDT_MakeID PRIMARY KEY,
    MakeName UDT_MakeName NOT NULL DEFAULT N'Unknown Make',
    MakeCountry UDT_MakeCountry DEFAULT '---'
);
GO


-- Model Table
CREATE TABLE Data.Model (
    ModelID UDT_ModelID PRIMARY KEY,
    MakeID UDT_MakeID FOREIGN KEY REFERENCES Data.Make(MakeID),
    ModelName UDT_ModelName DEFAULT N'Unknown Model',
);
GO


-- Customer Table
CREATE TABLE Data.Customer (
    CustomerID UDT_CustomerID PRIMARY KEY,
    CustomerName UDT_CustomerName NOT NULL DEFAULT N'Anonymous',
    Address1 UDT_Address DEFAULT N'Address Not Provided',
    Address2 UDT_Address DEFAULT N'Address Not Provided',
    Town UDT_Address DEFAULT N'Town not found',
    PostCode UDT_Address DEFAULT N'00000',
    CountryID UDT_CountryID FOREIGN KEY REFERENCES Data.Country(CountryID),
    IsReseller UDT_IsReseller DEFAULT 0,
    IsCreditRisk UDT_IsCreditRisk DEFAULT 0
);
GO


-- Sales Table
CREATE TABLE Data.Sales (
    SalesID UDT_SalesID PRIMARY KEY,
    CustomerID UDT_CustomerID FOREIGN KEY REFERENCES Data.Customer(CustomerID),
    InvoiceNumber UDT_InvoiceNumber DEFAULT '00000000',
    SaleDate UDT_SaleDate DEFAULT GETDATE(),
    TotalSalePrice UDT_TotalSalePrice DEFAULT 0
);
GO


-- Stock Table (foreign keys aligned)
CREATE TABLE Data.Stock (
    StockCode UDT_StockCode PRIMARY KEY,
    ModelID UDT_ModelID FOREIGN KEY REFERENCES Data.Model(ModelID),
    ColorID UDT_ColorID,  -- FK removed if Color table doesn't exist
    Cost UDT_Cost DEFAULT 0,
    RepairsCost UDT_RepairsCost DEFAULT 0,
    PartsCost UDT_PartsCost DEFAULT 0,
    TransportInCost UDT_TransportCost DEFAULT 0,
    BuyerComments UDT_BuyerComments DEFAULT N'',
    IsRHD UDT_IsReseller DEFAULT 0,
    DateBought UDT_DateBought DEFAULT GETDATE(),
    TimeBought UDT_TimeBought DEFAULT CAST(GETDATE() AS TIME(7))
);
GO


-- SalesDetails Table (foreign keys aligned)
CREATE TABLE Data.SalesDetails (
    SalesDetailsID INT PRIMARY KEY,
    SalesID UDT_SalesID FOREIGN KEY REFERENCES Data.Sales(SalesID),
    StockID UDT_StockCode,  -- FK removed if Color table doesn't exist
    LineItemNumber UDT_LineItemNumber DEFAULT 1,
    SalePrice UDT_SalePrice DEFAULT 0,
    LineItemDiscount UDT_LineItemDiscount DEFAULT 0
);
GO


In [ ]:
--INSERTING INTO NEW DATA(just testing it)
INSERT INTO Data.Country (CountryID, CountryName, CountryISO2, CountryISO3, SalesRegion)
VALUES 
(1, N'Belgium', N'BE', N'BEL', N'EMEA'),
(2, N'France', N'FR', N'FRA', N'EMEA'),
(3, N'Germany', N'DE', N'DEU', N'EMEA'),
(4, N'Italy', N'IT', N'ITA', N'EMEA'),
(5, N'Spain', N'ES', N'ESP', N'EMEA'),
(6, N'United Kingdom', N'GB', N'GBR', N'EMEA'),
(7, N'United States', N'US', N'USA', N'North America'),
(8, N'China', N'CN', N'CHN', N'Asia'),
(9, N'India', N'IN', N'IND', N'Asia'),
(10, N'Switzerland', N'CH', N'CHF', N'EMEA');
GO


In [ ]:
--DROP FOREIGN KEYS


SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

-- =============================================
-- Author: Nayem Sarker
-- Create date: 5/14/2025
-- Description:	Drop the Foreign Keys From the Star Schema
-- =============================================


DROP PROCEDURE IF EXISTS [Project2.5].[DropForeignKeysFromStarSchemaData]

GO

CREATE PROCEDURE [Project2.5].[DropForeignKeysFromStarSchemaData]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    Declare @ForeignKeyName VARCHAR(255)
    DECLARE @SQL VARCHAR(MAX)
    DECLARE @TableName VARCHAR(255)
    DECLARE ForeignKeyCursor CURSOR FOR 
    
    SELECT fk.name as ForeignKeyName,
        QUOTENAME(OBJECT_SCHEMA_NAME(fk.parent_object_id)) + '.'+t.name as TableName
    FROM sys.foreign_keys as fk
    INNER JOIN sys.tables as t on fk.parent_object_id = t.object_id

    OPEN ForeignKeyCursor

    FETCH NEXT FROM ForeignKeyCursor INTO @ForeignKeyName, @TableName

    WHILE @@FETCH_STATUS = 0 
    BEGIN
        SET @SQL = 'ALTER TABLE ' + @TableName + ' DROP CONSTRAINT IF EXISTS ' + @ForeignKeyName + ';'
        EXEC(@SQL)
        
        FETCH NEXT FROM ForeignKeyCursor INTO @ForeignKeyName, @TableName
    END

    CLOSE ForeignKeyCursor
    DEALLOCATE ForeignKeyCursor

    EXEC [Process].[usp_TrackWorkFlow]
        @WorkFlowStepDescription = 'Drop Foreign Keys.',
        @UserAuthorizationKey = @UserAuthorizationKey,
        @WorkFlowStepTableRowCount = -1;

END;
GO


In [ ]:
CREATE PROCEDURE CreateDbSecurityAndLoadUsers
AS
BEGIN
    -- Create schema if it doesn't exist
    IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'DbSecurity')
    BEGIN
        EXEC('CREATE SCHEMA DbSecurity');
    END

    -- Create the UserAuthorization table
    IF NOT EXISTS (
        SELECT * FROM sys.tables 
        WHERE name = 'UserAuthorization' 
        AND SCHEMA_NAME(schema_id) = 'DbSecurity'
    )
    BEGIN
        CREATE TABLE DbSecurity.UserAuthorization (
            UserAuthorizationKey INT NOT NULL PRIMARY KEY,
            ClassTime NCHAR(5) NULL CHECK (ClassTime IN ('10:45')),
            IndividualProject NVARCHAR(60) NULL DEFAULT 'PROJECT 2 RECREATE THE BICLASS DATABASE STAR SCHEMA',
            GroupMemberLastName NVARCHAR(35) NOT NULL,
            GroupMemberFirstName NVARCHAR(25) NOT NULL,
            GroupName NVARCHAR(20) NOT NULL,
            DateAdded DATETIME2 DEFAULT SYSDATETIME()
        );
    END

    -- Insert sample records
    INSERT INTO DbSecurity.UserAuthorization (
        UserAuthorizationKey,
        ClassTime,
        IndividualProject,
        GroupMemberLastName,
        GroupMemberFirstName,
        GroupName
    )
    VALUES
    (1, '10:45', 'created script for new database','Felix', 'Ashly', 'Group#3'),
    (2, '10:45', 'looked into normalization', 'Vathada', 'Mai', 'Group#3'),
    (3, '10:45', 'created udts', 'Mahir', 'Mehtab', 'Group#3'),
    (4, '10:45', 'created views', 'Wang', 'William', 'Group#3'),
    (5, '10:45', 'Creates dropping fks procedure', 'Sarker', 'Nayem', 'Group#3'),
    (6, '10:45', DEFAULT, 'Chen', 'Sabrina', 'Group#3'),
    (7, '10:45', 'default', 'default','default', 'default');
END;
GO

EXEC CreateDbSecurityAndLoadUsers;


In [ ]:
--INSERTING DATA FROM OLD DB TO NEW THATS NORMALIZED
--copy
INSERT INTO [PrestigeCars2.0].[Data].Make (MakeID, MakeName)
SELECT MakeID, MakeName
FROM PrestigeCars.[Data].Make
WHERE MakeID NOT IN (
    SELECT MakeID FROM [PrestigeCars2.0].[Data].Make
);

--now inset model
INSERT INTO [PrestigeCars2.0].[Data].Model (ModelID, MakeID, ModelName)
SELECT ModelID, MakeID, ModelName
FROM PrestigeCars.[Data].Model;


In [ ]:
--check for duplicates syntax
SELECT *
FROM YourTable
GROUP BY col1, col2, col3, ...
HAVING COUNT(*) > 1;

--actually checking Data.Model
SELECT *
FROM Data.Model
GROUP BY ModelID, MakeID, ModelName
HAVING COUNT(*) > 1;

In [ ]:
--Combining Sales Data Across Years 
--WILLIAM'S WORK

CREATE VIEW vw_AllSales AS 
SELECT * FROM DataTransfer.Sales2015 
UNION ALL 
SELECT * FROM DataTransfer.Sales2016 
UNION ALL 
SELECT * FROM DataTransfer.Sales2017 
UNION ALL 
SELECT * FROM DataTransfer.Sales2018; 

--Show car models, costs, and their marketing types 

CREATE VIEW vw_StockWithMarketing AS 
SELECT  
s.MakeName, 
s.ModelName, 
s.Cost, 
m.MarketingType 
FROM Output.StockPrices s 
LEFT JOIN Reference.MarketingCategories m  
ON s.MakeName = m.MakeName; 

--Show budget details grouped by year and month 

CREATE VIEW vw_MonthlyBudgetSummary AS 
SELECT  
Year, 
Month, 
SUM(BudgetValue) AS TotalBudget 
FROM Reference.Budget 
GROUP BY Year, Month; 

--View for Reference.Forex table 
CREATE VIEW vw_Forex AS  
SELECT  
ExchangeDate,  
ISOCurrency,  
ExchangeRate  
FROM Reference.Forex;  
GO 

--View for Marketing Categories table 
 
CREATE VIEW vw_MarketingCategories 
AS 
SELECT 
    MakeName, 
    MarketingType 
FROM Reference.MarketingCategories; 
GO 

--View for Marketing Information table 
 
CREATE VIEW vw_MarketingInformation 
AS 
SELECT  
    CUST, 
    Country, 
    SpendCapacity 
FROM Reference.MarketingInformation; 
GO 

--View for Sales Budgets table 
 
CREATE VIEW vw_SalesBudgets 
AS 
SELECT  
    BudgetArea, 
    BudgetAmount, 
    BudgetYear, 
    DateUpdated, 
    Comments, 
    BudgetMonth 
FROM Reference.SalesBudgets; 
GO 

--View for Sales Category table 
 
CREATE VIEW vw_SalesCategory 
AS 
SELECT  
    LowerThreshold, 
    UpperThreshold, 
    CategoryDescription 
FROM Reference.SalesCategory; 
GO 

--View for Staff table 
 
CREATE VIEW vw_Staff 
AS 
SELECT 
    StaffID, 
    StaffName, 
    ManagerID, 
    Department 
FROM Reference.Staff; 
GO 

--View for Staff Hierarchy table 
 
CREATE VIEW vw_StaffHierarchy 
AS 
SELECT 
    HierarchyReference, 
    StaffID, 
    StaffName, 
    ManagerID, 
    Department 
FROM Reference.StaffHierarchy; 
GO 

--View for Yearly Sales table 
 
CREATE VIEW vw_YearlySales 
AS 
SELECT 
    MakeName, 
    ModelName, 
    CustomerName, 
    CountryName, 
    Cost, 
    RepairsCost, 
    PartsCost, 
    TransportInCost, 
    SalePrice, 
    SaleDate 
FROM Reference.YearlySales; 
GO 

--View for Sales in Pounds table 
 
CREATE VIEW vw_SalesInPounds 
AS 
SELECT 
    MakeName, 
    ModelName, 
    VehicleCost 
FROM SourceData.SalesInPounds; 
GO 
 
--View for Sales Text table 
 
CREATE VIEW vw_SalesText 
AS 
SELECT 
    CountryName, 
    MakeName, 
    Cost, 
    SalePrice 
FROM SourceData.SalesText; 
GO 

In [ ]:
--ITVF
--WILLIAM'S WORK

--Get all sales from 2015 to 2018 
CREATE FUNCTION dbo.fn_GetAllSales() 
RETURNS TABLE 
AS 
RETURN 
( 
    SELECT *, 2015 AS SalesYear FROM DataTransfer.Sales2015 
    UNION ALL 
    SELECT *, 2016 AS SalesYear FROM DataTransfer.Sales2016 
    UNION ALL 
    SELECT *, 2017 AS SalesYear FROM DataTransfer.Sales2017 
    UNION ALL 
    SELECT *, 2018 AS SalesYear FROM DataTransfer.Sales2018 
); 

--Function to filter StockPrices by Make 
 
CREATE FUNCTION fn_StockPricesByMake (@MakeName NVARCHAR(100)) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        MakeName, 
        ModelName, 
        Cost 
    FROM Output.StockPrices 
    WHERE MakeName = @MakeName 
); 
GO

--Function to get Budget entries by Year/Month 
 
CREATE FUNCTION fn_BudgetByYearMonth ( 
    @Year INT, 
    @Month TINYINT 
) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        BudgetKey, 
        BudgetValue, 
        Year, 
        Month, 
        BudgetDetail, 
        BudgetElement 
    FROM Reference.Budget 
    WHERE Year = @Year 
      AND Month = @Month 
); 
GO 

--Function to filter by currency code 
 
CREATE FUNCTION fn_ForexByCurrency (@ISOCurrency CHAR(3)) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        ExchangeDate, 
        ISOCurrency, 
        ExchangeRate 
    FROM Reference.Forex 
    WHERE ISOCurrency = @ISOCurrency 
); 
GO 

--Filter by make name 
 
CREATE FUNCTION fn_MarketingCategoriesByMake (@MakeName NVARCHAR(100)) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        MakeName, 
        MarketingType 
    FROM Reference.MarketingCategories 
    WHERE MakeName = @MakeName 
); 
GO

--Filter by country code 
 
CREATE FUNCTION fn_MarketingInfoByCountry (@Country NCHAR(10)) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        CUST, 
        Country, 
        SpendCapacity 
    FROM Reference.MarketingInformation 
    WHERE Country = @Country 
); 
GO 

--Filter SalesBudgets by Year/Month combination 
 
CREATE FUNCTION fn_SalesBudgetsByYearMonth ( 
    @BudgetYear INT, 
    @BudgetMonth TINYINT 
) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        BudgetArea, 
        BudgetAmount, 
        BudgetYear, 
        DateUpdated, 
        Comments, 
        BudgetMonth 
    FROM Reference.SalesBudgets 
    WHERE BudgetYear = @BudgetYear 
      AND BudgetMonth = @BudgetMonth 
); 
GO

--Find SalesCategory by value threshold 
 
CREATE FUNCTION fn_SalesCategoryByValue (@Value INT) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        LowerThreshold, 
        UpperThreshold, 
        CategoryDescription 
    FROM Reference.SalesCategory 
    WHERE @Value BETWEEN LowerThreshold AND UpperThreshold 
); 
GO 

--Filter Staff by Department 
 
CREATE FUNCTION fn_StaffByDepartment (@Department NVARCHAR(50)) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        StaffID, 
        StaffName, 
        ManagerID, 
        Department 
    FROM Reference.Staff 
    WHERE Department = @Department 
); 
GO

--Filter StaffHierarchy by Manager 
 
CREATE FUNCTION fn_StaffHierarchyByManager (@ManagerID INT) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        HierarchyReference, 
        StaffID, 
        StaffName, 
        ManagerID, 
        Department 
    FROM Reference.StaffHierarchy 
    WHERE ManagerID = @ManagerID 
); 
GO 

--Filter YearlySales by date range 
 
CREATE FUNCTION fn_YearlySalesByDateRange ( 
    @StartDate DATETIME, 
    @EndDate DATETIME 
) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        MakeName, 
        ModelName, 
        CustomerName, 
        CountryName, 
        Cost, 
        RepairsCost, 
        PartsCost, 
        TransportInCost, 
        SalePrice, 
        SaleDate 
    FROM Reference.YearlySales 
    WHERE SaleDate BETWEEN @StartDate AND @EndDate 
); 
GO 

--Filter SalesInPounds by Make/Model 
 
CREATE FUNCTION fn_SalesInPoundsByMakeModel ( 
    @MakeName NVARCHAR(100), 
    @ModelName NVARCHAR(150) 
) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        MakeName, 
        ModelName, 
        VehicleCost 
    FROM SourceData.SalesInPounds 
    WHERE MakeName = @MakeName 
      AND ModelName = @ModelName 
); 
GO 

--Filter SalesText by Country and Make 
 
CREATE FUNCTION fn_SalesTextByCountryMake ( 
    @CountryName NVARCHAR(150), 
    @MakeName NVARCHAR(100) 
) 
RETURNS TABLE 
AS 
RETURN ( 
    SELECT  
        CountryName, 
        MakeName, 
        Cost, 
        SalePrice 
    FROM SourceData.SalesText 
    WHERE CountryName = @CountryName 
      AND MakeName = @MakeName 
); 
GO